In [1]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.11.0
TFX version: 1.12.0


In [2]:
import os

PIPELINE_NAME = "NNJZ"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

In [3]:
base_dir = os.getcwd()
data_dir = os.path.join(base_dir, "data/")
print(data_dir)

C:\Users\kvdte\My Drive (kvdtempel@gmail.com)\AILabs\JUP_AutoEncoder_JZ\data/


In [7]:
import tempfile
import os
import urllib.request

# Maak eerst een tijdelijke directory, verkrijg de dataset m.b.v. een Uri, en maakeen kopie naar de tijdelijke dir
# urllib.request.urlretrieve(url, filename=None, reporthook=None, data=None)
# Copy a network object denoted by a URL to a local file. If the URL points to a local file, the object 
# will not be copied unless filename is supplied. Return a tuple (filename, headers) where filename is the local file

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'file:///Users/kvdte/My Drive (kvdtempel@gmail.com)/AILabs/JUP_AutoEncoder_JZ/data/NN_Declaraties_autoencoder_train.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)
print(DATA_ROOT)

C:\Users\kvdte\AppData\Local\Temp\tfx-dataa2fkxv8x


In [16]:
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
import pandas as pd

# Create an instance of the InteractiveContext
context = InteractiveContext()

# Read the CSV file into a pandas DataFrame
data = pd.read_csv("data/NN_Declaraties_autoencoder_train.csv", decimal=",", sep=";")
data.to_csv('tfxdata/data.csv')

# Create an instance of the CsvExampleGen component
csv_example_gen = tfx.components.CsvExampleGen(input_base='tfxdata/')

# Get the examples from the CsvExampleGen component
# example = csv_example_gen.outputs['examples'].get()
context.run(csv_example_gen)

# Create an instance of the StatisticsGen component
statistics_gen = tfx.components.StatisticsGen(examples=csv_example_gen.outputs['examples'])

# Execute the components using the InteractiveContext
context.run(statistics_gen)

# Get the statistics from the InteractiveContext
statistics = statistics_gen.outputs['statistics'].get()

# Display the generated statistics
print(statistics)


INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Processing input csv data tfxdata/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to C:\Users\kvdte\AppData\Local\Temp\tfx-interactive-2023-02-13T22_19_37.568094-dz8d8vhu\StatisticsGen\statistics\2\Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to C:\Users\kvdte\Ap

AttributeError: 'InteractiveContext' object has no attribute 'get_component'

In [18]:
statistics = statistics_gen.outputs['statistics'].get()
print(statistics)

[Artifact(artifact: id: 2
type_id: 16
uri: "C:\\Users\\kvdte\\AppData\\Local\\Temp\\tfx-interactive-2023-02-13T22_19_37.568094-dz8d8vhu\\StatisticsGen\\statistics\\2"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "statistics:2023-02-13T22:22:43.671429"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "StatisticsGen"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.12.0"
  }
}
state: LIVE
name: "statistics:2023-02-13T22:22:43.671429"
, artifact_type: id: 16
name: "ExampleStatistics"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
base_type: STATISTICS
)]


In [ ]:
# Jupyter notebook Python code voor Autoencoder vraagstuk
# Auteur Kees van den Tempel, AI-labs BV, Tricht
# Versie 1.0
# 25-10-2022

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import sklearn
import datetime
from sklearn.preprocessing import OrdinalEncoder
from IPython.display import Image
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import os
####*IMPORANT*: Have to do this line *before* importing tensorflow
os.environ['PYTHONHASHSEED']=str(2)
import random as rd
import tensorflow as tf
import pickle
#%matplotlib inline

# lees de bestanden met afbeeldingen met digits in
x_train = pd.read_csv("data/NN_Declaraties_autoencoder_train.csv", decimal=",", sep=";")
x_test = pd.read_csv("data/NN_Declaraties_autoencoder_test.csv", decimal=",", sep=";")

print("De x_train dataset bevat " + str(x_train.shape) + " records")
print("De x_train dataset bevat " + str(x_train.isnull().values.sum()) + " Null-waarden\n")
print("De x_test dataset bevat " + str(x_test.shape) + " records")
print("De x_test dataset bevat " + str(x_test.isnull().values.sum()) + " Null-waarden\n")

pd.options.mode.chained_assignment = None

In [ ]:
# variabelen het verbeteren van de dataset

# variabelen voor het optimaliseren van het algoritme
HyperparameterTuning = False
# Boolean om te bepalen of kFold wordt gebruikt
UseKFold = False
# Aantal kFold iteraties
kKfold = 6
DevSet = 0

# Tensorboards uitproberen
# Dropout = 0.1
# Bottleneck = helft van de input layer units of percentage (zeg 80%) van oorspronkelijke units

In [ ]:
x_train.info(verbose=1)

In [ ]:
# Execute onderstaande statement in de anaconda terminal
# python -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"
# Gaf foutmelding dat er bepaalde dlls niet gevonden konden worden
# Computer vaak herstarten na elke installatie
# Niet vindbare dll's in de map c:\windows\system32 kopieren gaf de oplossing

# Python version:  3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
# Tensorflow version:  2.10.0
# Keras version:  2.10.0
# Cudnn version:  64_8
# Cuda version:  64_112
# Eager mode:  True
# GPU is available
# Num GPUs: 1
# 1 Physical GPUs, 1 Logical GPUs

from __future__ import absolute_import, division, print_function, unicode_literals
import sys
import numpy as np
import tensorflow as tf
import tensorflow.python.platform.build_info as build
from tensorflow import keras

version = tf.__version__
executing_eagerly = tf.executing_eagerly()
available = tf.config.experimental.list_physical_devices("GPU")
physical_devices = tf.config.list_physical_devices('GPU')

print("Python version: ", sys.version)
print("Tensorflow version: ", version)
print("Keras version: ", keras.__version__)
print("Cudnn version: ", build.build_info['cudnn_version'])
print("Cuda version: ", build.build_info['cuda_version'])
print("Eager mode: ", executing_eagerly)
print("GPU is", "available" if available else "NOT AVAILABLE")
print("Num GPUs:", len(physical_devices))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
# init deterministic seed
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(2)
   tf.random.set_seed(2)
   np.random.seed(2)
   rd.seed(2)


In [ ]:
pd.set_option('display.max_columns', None)
x_train.head(10)

In [ ]:
def DeleteVars(df):
    df.drop(['Unnamed: 0', 'DeclaratieId'], axis=1, inplace=True)

    volgnummers = [var for var in df.columns if '__' in var]
    print(volgnummers)
    df.drop(volgnummers, axis=1, inplace=True)

    #gemeenten = [var for var in df.columns if 'Gemeente' in var]
    #print(gemeenten)
    #df.drop(gemeenten, axis=1, inplace=True)

    #client = [var for var in df.columns if 'Client' in var]
    #print(client)
    #df.drop(client, axis=1, inplace=True)

    contract = [var for var in df.columns if 'ContractId' in var]
    print(contract)
    df.drop(contract, axis=1, inplace=True)

DID_train = x_train['DeclaratieId']
DID_test = x_test['DeclaratieId']
DeleteVars(x_train)
DeleteVars(x_test)

In [ ]:
# Splits de digits-dataset in een training- en een testset voor het machine learning model
# Eerst moet er een training en test set gemaakt worden om daarna allerlei bewerkingen en filters toe te passen
# om zo data leakage te voorkomen
   
print(str(type(x_train)) + ': x_train Shape:', x_train.shape)
print(str(type(x_test)) + ': x_test Shape:', x_test.shape)

In [ ]:
#Schakel over van panda.dataframes naar numpy arrays
x_train = np.array(x_train)
x_test = np.array(x_test)

print ("X-train (de kenmerken)" + str(x_train.shape))
print ("X-test (de test kenmerken)" + str(x_test.shape))

In [ ]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and dev-sets
if DevSet > 0:
    x_train, x_dev = train_test_split(x_train, test_size = DevSet, random_state = 42)
    print('Dimensie van de trainingset x_train:' + str (x_train.shape))
    print('Dimensie van de testset x_test:' + str(x_test.shape))
    print('Dimensie van de dev-set x_dev:' + str(x_dev.shape) + '\n')

In [ ]:
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import matplotlib

# DIT ZIJN METRICS VOOR CLASSIFICATION PROBLEMS
# https://medium.com/analytics-vidhya/evaluation-metrics-for-classification-problems-with-implementation-in-python-a20193b4f2c3
# https://www.kdnuggets.com/2018/04/right-metric-evaluating-machine-learning-models-1.html
# https://neptune.ai/blog/performance-metrics-in-machine-learning-complete-guide
# https://neptune.ai/blog/f1-score-accuracy-roc-auc-pr-auc

def DisplayMetrics(rf, y_test, y_rfpred, makeplots, titel):
    print("\n" + titel)
    print("Accuracy (goed-voorspeld/totaal) (niet voor unbalanced dataset): " + str(accuracy_score(y_test, y_rfpred).round(3)))
    print(f"Precision Score is (TP/TP+FP): {precision_score(y_test, y_rfpred).round(3)}")
    print(f"Recall Score is (TP/TP+FN): {recall_score(y_test, y_rfpred).round(3)}")
    print(f"F1 Score is (2*P*R/P+R): {f1_score(y_test, y_rfpred).round(3)}")    
    print("Random Forests roc-auc (bin class): {}".format(roc_auc_score(y_test, y_rfpred).round(3)))

    if makeplots == True:
        matplotlib.rcParams.update(matplotlib.rcParamsDefault)
        
        # confusion_matrix function a matrix containing the summary of predictions
        cm = confusion_matrix(y_test, y_rfpred, labels=rf.classes_)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rf.classes_)
        disp.plot()
        plt.show()

        # Plotting the ROC
        fpr, tpr, threshold = roc_curve(y_test, y_rfpred)
        roc_auc = auc(fpr, tpr)
        plt.title('Receiver Operating Characteristic')
        plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
        plt.legend(loc = 'lower right')
        plt.plot([0, 1], [0, 1],'r--')
        plt.xlim([0, 1])
        plt.ylim([0, 1])
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        plt.show()
        
    # FOR DEEP LEARNING CLASSIFICATION PROBLEMS
    # print("\nCLASSIFICATION METRICS")
    # print(keras.losses.categorical_crossentropy(y_test, y_pred, from_logits=False, label_smoothing=0))
    # print("\nCATEGORICAL ACCURACY")
    # print("Keras Accuracy: " + str(keras.metrics.categorical_accuracy(y_test, y_pred)))
    # print(keras.metrics.top_k_categorical_accuracy(y_test, y_pred, k=5))
    # x_pred_max = np.argmax(x_pred, axis=1)
    # x_test_max = np.argmax(x_test, axis=1)
    # score = accuracy_score(x_test_max, x_pred_max)
    # print ("\nAccuracy argmax: " + str(score))


In [ ]:
# NEURAAL NETWERK
# Train dezelfde gegevens met een eenvoudig neuraal netwerk
# Dit is een multiclassification probleem. De softmax functie  levert de kans op (in vector vorm) dat 
# de test-waardes op een hot-encoded (werkelijke) vector lijken. Dit wordt gemeten
# met cross-entropy: https://www.youtube.com/watch?v=tRsSi_sqXjI
#
# LEES DIT : https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks
# EN DIT: https://machinelearningmastery.com/understand-the-dynamics-of-learning-rate-on-deep-learning-neural-networks/
# https://medium.com/@mgazar/lenet-5-in-9-lines-of-code-using-keras-ac99294c8086
# https://www.pyimagesearch.com/2016/08/01/lenet-convolutional-neural-network-in-python/
# 
# --- LeNet implementatie ---
# Convolution #1. Input = 32x32x1. Output = 28x28x6 conv2d
# SubSampling #1. Input = 28x28x6. Output = 14x14x6. SubSampling is simply Average Pooling so we use avg_pool
# Convolution #2. Input = 14x14x6. Output = 10x10x16 conv2d
# SubSampling #2. Input = 10x10x16. Output = 5x5x16 avg_pool
# Fully Connected #1. Input = 5x5x16. Output = 120
# Fully Connected #2. Input = 120. Output = 84
# Output 10

# https://machinelearningmastery.com/understand-the-dynamics-of-learning-rate-on-deep-learning-neural-networks/
# https://machinelearningmastery.com/how-to-reduce-overfitting-in-deep-learning-with-weight-regularization/

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM, BatchNormalization, Conv2D, Input, MaxPool2D,Flatten, Permute, GlobalAveragePooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Nadam
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras import initializers, activations
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import CosineSimilarity
from tensorflow.keras import regularizers
from keras.losses import mean_squared_error
from keras.layers import LeakyReLU

def SelectOptimizer(i, lr):
    if i==1:
        tmp = Adam(learning_rate=lr, epsilon=1e-07)
    elif i==2:
        tmp = Nadam(learning_rate=lr, epsilon=1e-07)
    elif i==3:
        tmp = SGD(learning_rate=lr, momentum=0.8, nesterov=False)
    elif i==4:
        tmp = SGD(learning_rate=lr, momentum=0.8, nesterov=True)
    elif i==5:
        tmp = RMSprop(learning_rate=lr, momentum=0.8, epsilon=1e-07)
    return tmp

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis = None))


def NeuralNetworkModel(activation, Param1, Param2, Param3, Param4, Param5, reg, DropoutVal, lr, OptIndex):
    # Selecteer een optimizer
    opt = SelectOptimizer(OptIndex, lr)
    # tf.random.set_seed(1234)
    
    # Script gaf een foutmelding: onderstaande om foutmelding te verhelpen
    # https://stackoverflow.com/questions/73978774/how-to-get-rid-of-the-userwarning-the-initializer-glorotuniform-is-unseeded-m
    #initializer = initializers.RandomNormal(mean=0.0, stddev=0.5)
    #initializer = initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None)
    #initializer = initializers.Zeros()
    #initializer = initializers.Ones()
    #initializer = initializers.GlorotNormal()
    initializer = initializers.GlorotUniform(seed=0)    #Xavier = default
    #initializer = initializers.Identity()
    
    kregularizer=regularizers.L1(reg)
    #kregularizer=l1_l2(l1=0.00001, l2=0.0001),
    bregularizer=l2(1e-4),
    aregularizer=l2(1e-5)
    #callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

    # INPUT LAYER 1: default Param1 = 82
    model = tf.keras.models.Sequential()    
    #model.add(tf.keras.Input(shape=(x_train.shape[1],)))
    
    # HIDDEN LAYER 1: default Param1 = 60
    if Param1>0:
        if reg>0:
             model.add(tf.keras.layers.Dense(units=Param1, kernel_regularizer=l2(reg), kernel_initializer=initializer))
        else:
             model.add(tf.keras.layers.Dense(units=Param1, kernel_initializer=initializer))
        model.add(Activation(activation))
        if DropoutVal>0:
            model.add(Dropout(DropoutVal))
    
    # HIDDEN LAYER 2: default Param2 = 42
    if reg>0:
         model.add(tf.keras.layers.Dense(units=Param2, kernel_regularizer=l2(reg), kernel_initializer=initializer))
    else:
         model.add(tf.keras.layers.Dense(units=Param2, kernel_initializer=initializer))
    model.add(Activation(activation))
    if DropoutVal>0:
        model.add(Dropout(DropoutVal))
    
    # HIDDEN Layer 3: default Param3 = 21
    if reg>0:
         model.add(tf.keras.layers.Dense(units=Param3, kernel_regularizer=l2(reg), kernel_initializer=initializer))
    else:
         model.add(tf.keras.layers.Dense(units=Param3, kernel_initializer=initializer))
    model.add(Activation(activation))
    if DropoutVal>0:
        model.add(Dropout(DropoutVal))
    
    if Param4>0:
        if reg>0:
             model.add(tf.keras.layers.Dense(units=Param4, kernel_regularizer=l2(reg), kernel_initializer=initializer))
        else:
             model.add(tf.keras.layers.Dense(units=Param4, kernel_initializer=initializer))
        model.add(Activation(activation))
        if DropoutVal>0:
            model.add(Dropout(DropoutVal))

    # MID HIDDEN Layer 5: default Param5 = 10
    if reg>0:
         model.add(tf.keras.layers.Dense(units=Param5, kernel_regularizer=l2(reg), kernel_initializer=initializer))
    else:
         model.add(tf.keras.layers.Dense(units=Param5, kernel_initializer=initializer))
    model.add(Activation(activation))
    if DropoutVal>0:
        model.add(Dropout(DropoutVal))

    if Param4>0:
        if reg>0:
             model.add(tf.keras.layers.Dense(units=Param4, kernel_regularizer=l2(reg), kernel_initializer=initializer))
        else:
             model.add(tf.keras.layers.Dense(units=Param4, kernel_initializer=initializer))
        model.add(Activation(activation))
        if DropoutVal>0:
            model.add(Dropout(DropoutVal))
            
    # HIDDEN Layer 3: default Param3 = 21
    if reg>0:
         model.add(tf.keras.layers.Dense(units=Param3, kernel_regularizer=l2(reg), kernel_initializer=initializer))
    else:
         model.add(tf.keras.layers.Dense(units=Param3, kernel_initializer=initializer))
    model.add(Activation(activation))
    if DropoutVal>0:
        model.add(Dropout(DropoutVal))

    # HIDDEN LAYER 2: default Param2 = 42
    if reg>0:
         model.add(tf.keras.layers.Dense(units=Param2, kernel_regularizer=l2(reg), kernel_initializer=initializer))
    else:
         model.add(tf.keras.layers.Dense(units=Param2, kernel_initializer=initializer))
    model.add(Activation(activation))
    if DropoutVal>0:
        model.add(Dropout(DropoutVal))
    
    # OUTPUT layer: default Param1 = 84
    if Param1>0:
        if reg>0:
             model.add(tf.keras.layers.Dense(units=Param1, kernel_regularizer=l2(reg), kernel_initializer=initializer))
        else:
             model.add(tf.keras.layers.Dense(units=Param1, kernel_initializer=initializer))
        model.add(Activation('linear'))
        if DropoutVal>0:
            model.add(Dropout(DropoutVal))
       
    # model.compile(optimizer=opt, loss=root_mean_squared_error, metrics=['mse', 'mae', 'mape', 'accuracy'])    
    model.compile(optimizer=opt, loss=root_mean_squared_error, metrics=['mse', 'mae', 'mape', 'accuracy'])    
    return model


In [ ]:
# def NeuralNetworkModel(Activation, Param1, Param2, Param3, Param4, Reg, DropoutVal, lr, Optimizer):
# HEEL GOED (99553 : 54228) (zonder Contract en Volgnummer) (batchsize=16384)
# met 250 EPOCHS lr=0.015, decay=0.05, momentum=0.8: 
# Correlatie > 0.7 (bestand2) en verrekende contracten NIET weggegooid (bestand 1)

print(x_train.shape)
# x_train.drop_duplicates(keep='first', inplace=True)
# print(x_train.shape)

# repetitions = (2,1)
# x_train = np.tile(x_train, repetitions)
# print(x_train.shape)

# define the learning rate change 
epochs = 10
learning_rate = 0.008 # initial learning rate
# decay_rate = learning_rate / epochs
decay_rate = 0.05
momentum = 0.8
Reg = 0.0001
DropoutVal = 0

InnerLayer1 = x_train.shape[1]
InnerLayer2 = 50
InnerLayer3 = 25
InnerLayer4 = 0
InnerLayer5 = 0
InnerLayer6 = 0
InnerLayer7 = 0
LatentLayer = 10

opt = SelectOptimizer(2, 0.01)
kregularizer=regularizers.L2(0.01)
bSize = 16384
initializer = initializers.GlorotUniform(seed=0)


In [ ]:
# Define the Required Callback Functions
from keras.callbacks import LearningRateScheduler

def exp_decay(epoch):
    #lrate = learning_rate * np.exp(-decay_rate*float(epoch))          #exponentieel
    lrate = learning_rate * 1/(1 + decay_rate * epoch)                #assymptoot naar 0
    #lrate = (((0.002-learning_rate)/epochs) * epoch) + learning_rate   #lineair naar 0
    return lrate

lr_rate = LearningRateScheduler(exp_decay)

class StopTraining(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        # if(logs.get('accuracy')>0.99) or (logs.get('val_loss')<0.001):
        if (logs.get('val_loss')<0.001):
            print("\n99% val_accuracy bereikt, dus algorithme stopt met trainen")
            self.model.stop_training = True


# een grafiek maken van de learning rate\
x = list(range(0,epochs))
y = []
for i in x: y.append(exp_decay(i))
fig, axs = plt.subplots(figsize=(8, 4))
axs.scatter(x, y)
axs.set_title('Learning rate als functie van epochs')
axs.set(xlabel='Epoch', ylabel='Learning rate')
plt.show() 


In [ ]:
# Voer neuraal netwerk uit   
# https://www.deeplearning.ai/ai-notes/initialization/
# loss: 0.0042 - accuracy: 0.9993 - val_loss: 0.0107 - val_accuracy: 0.9924
# NeuralNetworkModel("relu", 64, 32, 16, 8, 140, 0.0001, 0)
#
# OVERFITTING VOORKOMEN
# Activity Regularization: Penalize the model during training base on the magnitude of the activations.
# Weight Constraint: Constrain the magnitude of weights to be within a range or below a limit.
# Dropout: Probabilistically remove inputs during training.
# Noise: Add statistical noise to inputs during training.
# Early Stopping: Monitor model performance on a validation set and stop training when performance degrades.

# Veranderende learning rate instellen via een callback functie in Keras
# Ook wordt er een plot gemaakt van de learning rate
# https://machinelearningmastery.com/using-learning-rate-schedules-deep-learning-models-python-keras/
# https://towardsdatascience.com/simple-guide-to-hyperparameter-tuning-in-neural-networks-3fe03dad8594

# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
# lr_callback_1 = tf.keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", patience = 2, mode = "auto", min_delta = 0.01, cooldown = 0, min_lr = 0)
# tensorboard_callback_1 = tf.keras.callbacks.TensorBoard(log_dir = "data", histogram_freq = 1)
# modelckpt_callback_1 = tf.keras.callbacks.ModelCheckpoint(
#    monitor = "val_accuracy",
#    filepath = "data",
#    verbose = 1,
#    mode = "max",
#    save_weights_only = False,
#    save_best_only = True,
#    save_freq = "epoch"
#    )

import matplotlib.pyplot as plt
from keras.callbacks import History
from sklearn.utils import compute_class_weight
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot
from keras.layers import LeakyReLU

with tf.device("cpu:0"):

    # Autoencoder
    print("\nDefinieer AutoEncoder Neuraal netwerk")

    reset_random_seeds()
    if 1==2:
        NNmodel = tf.keras.models.Sequential() 
        NNmodel.add(tf.keras.Input(shape=(InnerLayer1,)))
        if InnerLayer2>0:
            NNmodel.add(tf.keras.layers.Dense(units=InnerLayer2, activation="relu", kernel_regularizer=l2(Reg)))
            if DropoutVal>0: NNmodel.add(Dropout(DropoutVal))
        if InnerLayer3>0:
            NNmodel.add(tf.keras.layers.Dense(units=InnerLayer3, activation="relu", kernel_regularizer=l2(Reg)))
            if DropoutVal>0: NNmodel.add(Dropout(DropoutVal))
        if InnerLayer4>0:
            NNmodel.add(tf.keras.layers.Dense(units=InnerLayer4, activation="relu", kernel_regularizer=l2(Reg)))
            if DropoutVal>0: NNmodel.add(Dropout(DropoutVal))
        if InnerLayer5>0:
            NNmodel.add(tf.keras.layers.Dense(units=InnerLayer5, activation="relu", kernel_regularizer=l2(Reg)))
            if DropoutVal>0: NNmodel.add(Dropout(DropoutVal))
        if InnerLayer6>0:
            NNmodel.add(tf.keras.layers.Dense(units=InnerLayer6, activation="relu", kernel_regularizer=l2(Reg)))
            if DropoutVal>0: NNmodel.add(Dropout(DropoutVal))
        if InnerLayer7>0:
            NNmodel.add(tf.keras.layers.Dense(units=InnerLayer7, activation="relu", kernel_regularizer=l2(Reg)))
            if DropoutVal>0: NNmodel.add(Dropout(DropoutVal))
        NNmodel.add(tf.keras.layers.Dense(units=LatentLayer, activation="relu", kernel_regularizer=l2(Reg)))
        if DropoutVal>0: NNmodel.add(Dropout(DropoutVal))
        if InnerLayer7>0:
            NNmodel.add(tf.keras.layers.Dense(units=InnerLayer7, activation="relu", kernel_regularizer=l2(Reg)))
            if DropoutVal>0: NNmodel.add(Dropout(DropoutVal))
        if InnerLayer6>0:
            NNmodel.add(tf.keras.layers.Dense(units=InnerLayer6, activation="relu", kernel_regularizer=l2(Reg)))
            if DropoutVal>0: NNmodel.add(Dropout(DropoutVal))
        if InnerLayer5>0:
            NNmodel.add(tf.keras.layers.Dense(units=InnerLayer5, activation="relu", kernel_regularizer=l2(Reg)))
            if DropoutVal>0: NNmodel.add(Dropout(DropoutVal))
        if InnerLayer4>0:
            NNmodel.add(tf.keras.layers.Dense(units=InnerLayer4, activation="relu", kernel_regularizer=l2(Reg)))
            if DropoutVal>0: NNmodel.add(Dropout(DropoutVal))
        if InnerLayer3>0:
            NNmodel.add(tf.keras.layers.Dense(units=InnerLayer3, activation="relu", kernel_regularizer=l2(Reg)))
            if DropoutVal>0: NNmodel.add(Dropout(DropoutVal))
        if InnerLayer2>0:
            NNmodel.add(tf.keras.layers.Dense(units=InnerLayer2, activation="relu", kernel_regularizer=l2(Reg)))
            if DropoutVal>0: NNmodel.add(Dropout(DropoutVal))
        NNmodel.add(tf.keras.layers.Dense(units=InnerLayer1, activation="linear", kernel_regularizer=l2(Reg)))
        opt = Nadam(learning_rate=0.01, epsilon=1e-07)
        NNmodel.compile(optimizer=opt, loss=root_mean_squared_error, metrics=['mse', 'mae', 'mape', 'accuracy'])    
        print(NNmodel.summary())
    else:
        NNmodel = NeuralNetworkModel("relu", InnerLayer1, InnerLayer2, InnerLayer3, InnerLayer4, LatentLayer, 0, DropoutVal, 0.01, 2)

    # learning schedule callback
    callbacks_list = [lr_rate, StopTraining()]

    # hist = NNmodel.fit(x_train, x_train, batch_size=8192, epochs=20, validation_data=(x_test, x_test), callbacks = [StopTraining(), lr_callback_1, tensorboard_callback_1, modelckpt_callback_1], max_queue_size = 20, use_multiprocessing = True, workers = 24, verbose=1)
    hist = NNmodel.fit(x_train, x_train, batch_size=bSize, epochs=epochs, validation_data=(x_test, x_test), callbacks=callbacks_list, verbose=1)
    print(NNmodel.summary())
    
    # plot the autoencoder
    plot_model(NNmodel, 'data/autoencoder.png', show_shapes=True)    

    x_pred = NNmodel.predict(x_test)

    # plot loss and accuracy during training
    fig, axs = plt.subplots(2,1,figsize=(10, 10))
    axs[0].plot(hist.history['loss'], label='train')
    axs[0].plot(hist.history['val_loss'], label='test')
    axs[0].set_title('Loss')
    axs[0].set(xlabel='Epoch', ylabel='Loss')
    axs[0].legend()

    axs[1].plot(hist.history['mse'], label='train-mse')
    axs[1].plot(hist.history['mae'], label='train-mae')
    axs[1].plot(hist.history['accuracy'], label='train-accuracy')
    axs[1].plot(hist.history['val_mse'], label='test-mse')
    axs[1].plot(hist.history['val_mae'], label='test-mae')
    axs[1].plot(hist.history['val_accuracy'], label='test-accuracy')
    axs[1].set_title('Mse / Mae / Accuracy')
    axs[1].set(xlabel='Epoch', ylabel='Mse / Mae / Accuracy')
    axs[1].legend()
    plt.show() 
    
    # save the weights of the model to file
    NNmodel.save('data/encoder.h5')
    
    print(hist.history.keys())

In [ ]:
import scipy
from sklearn.metrics import r2_score

print(list(hist.history['mse'])[-1])
print(list(hist.history['val_mse'])[-1])
print(list(hist.history['mae'])[-1])
print(list(hist.history['val_mae'])[-1])
print(list(hist.history['accuracy'])[-1])
print(list(hist.history['val_accuracy'])[-1])
print(list(hist.history['mape'])[-1])
print(list(hist.history['val_mape'])[-1])
print(list(hist.history['loss'])[-1])
print(list(hist.history['val_loss'])[-1])
print(list(hist.history['lr'])[-1])
print("\n")

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(range(1,10), list(hist.history['loss'])[-9:])
print("Slope: " + str(slope))

# Calculate Eucledian Distance
m = np.sqrt(np.sum(np.square(np.subtract(x_pred, x_test)), axis=1))
count = np.count_nonzero(m < 1)
print(str(len(x_test)) + " : " + str(count))

r2 = r2_score(x_test, x_pred)
print(r2)

In [ ]:
# Slope: -0.0002583066001534462
# 61587 : 60183
# 0.8023598236234031

In [ ]:
DF = pd.concat([DID_test, pd.DataFrame(m)], axis=1)
DF.head()
print(DF[DF[0]>2]['DeclaratieId'])

In [ ]:
NoBins = 8
pdm=pd.DataFrame(m)
plt.hist(pdm[pdm[0]<NoBins], bins=NoBins)
plt.show()

In [ ]:
# Pas het neurale netwerk nogmaals toe, maar nu door de training en test data 
# steeds te splitsen mbv kFold
import os
import scipy
from sklearn.model_selection import KFold
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from sklearn.metrics import r2_score

# https://stackoverflow.com/questions/39758094/clearing-tensorflow-gpu-memory-after-model-execution
from numba import cuda 
#InteractiveSession.close()

TF_CONFIG = ConfigProto()
TF_CONFIG.gpu_options.per_process_gpu_memory_fraction = 0.333
# TF_CONFIG = ConfigProto(gpu_options=GPUOptions(per_process_gpu_memory_fraction=0.1),allow_soft_placement=True)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

#gpu_options = tf.GPUOptions(allow_growth=True)
#session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
#InteractiveSession.close(_self)
# df = x_train[["ClientId", "Periodenummer", "GedeclareerdeKostenExclBtw", "GemeenteId", "nProductID", "nAanbiederID", "EerdereDeclaraties"]]


if HyperparameterTuning == True:
    with tf.device("cpu:0"):
        print("Dimensie van de X-training: " + str(np.shape(x_train)) + " "+ str(type(x_train)))
        print("Dimensie van de X-test: " + str(np.shape(x_test)) + " "+ str(type(x_test)))

        print("\nOptimalisatie beste Neurale Netwerk: aantal nodes per layer en met kFold)")
        NNscores=[]
        ParamScores=[]
        ParamScores.append(["Cnt", "LatentLay", "Dropout", "Param2", "lRate", "DevSet", "Reg", "Opt", "bSize", "Epochs", "Count_train", "Count_test", "R2_train", "R2_test", "x_train", "x_test", "vmse", "vmsetest", "vmae", "vmaetest", "vacc", "vacctest", "vloss", "vlosstest", "vmape", "vmapetest", "Slope"])
        cnt = 0
        Dout = DropoutVal
        DevSet = 0
        if DevSet>0:
            X_train, X_dev = train_test_split(x_train, test_size = DevSet, random_state = 42)
        else:
            X_train = x_train.copy()
        
        for LearnRate in range(0, 20, 20):
            #print("Optimalisatie Param 1; Learning Rate: " + str(float(LearnRate/100000)))
            lRate = LearnRate/100000
            # for dSet in range(1, 2, 1):
                # DevSet = dSet/40
                # X_train, X_dev = train_test_split(x_train, test_size = DevSet, random_state = 42)
            for DropoutV in range(10, 100, 10):
                Dout = float(DropoutV/100)
                print("Dropout: " + str(Dout) )
                for iReg in range(0,1,1):
                    Reg = iReg/100000
                    for Opt in range(2,3,1):
                        for Param2 in range(170,171,1):
                            bSize = 16*1024
                            for LatentLay in range(16,17,1):
                                #InteractiveSession.close(_self)
                                sess = InteractiveSession(config=TF_CONFIG)
                                reset_random_seeds()

                                # def NeuralNetworkModel(Activation, Param1, Param2, Param3, Param4, Param5, Reg, DropoutVal, lr, Optimizer):
                                NNmodel = NeuralNetworkModel("relu", InnerLayer1, InnerLayer2, InnerLayer3, InnerLayer4, LatentLay, Reg, Dout, lRate, Opt)
                                cnt=cnt+1
                                NNscores.clear()
                                print("\nParameters bij onderstaande resultaten\nLearningRate:" + str(lRate) + "  Dropout:" + str(Dout) + "  Param2:" + str(Param2) + "  LatentLayer:" + str(LatentLay) + "  DevSet:" + str(DevSet) + "  Reg:" + str(Reg) + "  Opt:" + str(Opt) + "  Batchsize:" + str(bSize) + "  Epochs:" + str(epochs))

                                hist = NNmodel.fit(X_train, X_train, batch_size=bSize, epochs=epochs, validation_data=(x_test, x_test), callbacks=callbacks_list, verbose=0)

                                vmse = list(hist.history['mse'])[-1]
                                vmsetest = list(hist.history['val_mse'])[-1]
                                vmae = list(hist.history['mae'])[-1]
                                vmaetest = list(hist.history['val_mae'])[-1]
                                vacc = list(hist.history['accuracy'])[-1]
                                vacctest = list(hist.history['val_accuracy'])[-1]
                                vloss = list(hist.history['loss'])[-1]
                                vlosstest = list(hist.history['val_loss'])[-1]
                                vmape = list(hist.history['mape'])[-1]
                                vmapetest = list(hist.history['val_mape'])[-1]
                                vlr = list(hist.history['lr'])[-1]

                                xtrain_pred = NNmodel.predict(X_train)
                                xtest_pred = NNmodel.predict(x_test)

                                # Bepaal de R2 score op de training set en bepaals hoeveel records een EUCL DIST < 1 hebben
                                m_train = np.sqrt(np.sum(np.square(np.subtract(xtrain_pred, X_train)), axis=1))
                                count_train = np.count_nonzero(m_train < 1)/X_train.shape[0]
                                r2_train = r2_score(X_train, xtrain_pred)
                                
                                # Bepaal de R2 score op de test set en bepaals hoeveel records een EUCL DIST < 1 hebben
                                m_test = np.sqrt(np.sum(np.square(np.subtract(xtest_pred, x_test)), axis=1))
                                count_test = np.count_nonzero(m_test < 1)/x_test.shape[0]
                                r2_test = r2_score(x_test, xtest_pred)
                                
                                slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(range(1,10), list(hist.history['loss'])[-9:])

                                print ("Loss: " + str(vloss) + "  Loss test-set: " + str(vlosstest))
                                ParamScores.append([cnt, LatentLay, Dout, Param2, vlr, DevSet, Reg, Opt, bSize, epochs, count_train, count_test, r2_train, r2_test, X_train.shape, x_test.shape, vmse, vmsetest, vmae, vmaetest, vacc, vacctest, vloss, vlosstest, vmape, vmapetest, slope])

                                #device = cuda.get_current_device()
                                #device.reset()
                                sess.close()
                                #cuda.select_device(0)
                                #cuda.close()

    fig, axs = plt.subplots(2,1,figsize=(10, 5))
    axs[0].plot(hist.history['loss'], label='train')
    axs[0].plot(hist.history['val_loss'], label='test')
    axs[0].set_title('Loss')
    axs[0].set(xlabel='epoch', ylabel='loss')
    axs[0].legend()

    axs[1].plot(hist.history['mse'], label='train-mse')
    axs[1].plot(hist.history['mae'], label='train-mae')
    axs[1].plot(hist.history['accuracy'], label='train-accuracy')
    axs[1].plot(hist.history['val_mse'], label='test-mse')
    axs[1].plot(hist.history['val_mae'], label='test-mae')
    axs[1].plot(hist.history['val_accuracy'], label='test-accuracy')
    axs[1].set_title('Mse / Mae / Accuracy')
    axs[1].set(xlabel='Epoch', ylabel='Mse / Mae / Accuracy')
    axs[1].legend()
    plt.show() 


In [ ]:
if HyperparameterTuning == True:
    df = pd.DataFrame(ParamScores)
    writer = pd.ExcelWriter('test_l2.xlsx', engine='xlsxwriter')
    df.to_excel(writer, sheet_name='testresults', index=False)
    writer.save()
    writer.close()
